In [1]:
from math import sqrt
import copy
import  traceback
import shutil
import random

import numpy as np  # linear algebra
import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from pydicom.uid import UID
from PIL import Image
from tqdm import tqdm
import openpyxl

In [2]:
def load_scan(path):
    slices = []
    for s in os.listdir(path):
        if not s.endswith('.dcm'):
            continue
        sl = pydicom.dcmread(os.path.join(path, s))
        try:
            sl_p = sl.pixel_array
        except AttributeError:
            traceback.print_exc()
            print(f'\tDelete {os.path.join(path, s)}')
            os.remove(os.path.join(path, s))
        else:
            slices.append(sl)
    slices.sort(key=lambda x: float(x.InstanceNumber))
    return slices, slices[0].SliceThickness

In [3]:
lower_b, upper_b = -100, 500

In [7]:
root = '/nfs3-p1/zsxm/dataset/2021-9-17-negative'
not_2_list = []
for patient in sorted(os.listdir(root)):
    patient = os.path.join(root, patient)
    if os.path.isfile(patient):
        continue
    if len(os.listdir(patient)) > 2:
        not_2_list.append(patient)
        continue
    ct, ct_thickness = load_scan(os.path.join(patient, '1'))
    try:
        cta, cta_thickness = load_scan(os.path.join(patient, '2'))
    except FileNotFoundError:
        print(patient, 'not have 2')
        continue
    if ct_thickness % cta_thickness != 0:
        print(patient, ct_thickness, cta_thickness)

/nfs3-p1/zsxm/dataset/2021-9-17-negative/dujinlian-17-20-36-82 5 3
/nfs3-p1/zsxm/dataset/2021-9-17-negative/huangxiang-22-26-42-84 5.000000 0.6250010729
/nfs3-p1/zsxm/dataset/2021-9-17-negative/luweiqing-16-20-37-93 5.000000 3
/nfs3-p1/zsxm/dataset/2021-9-17-negative/wanghaifang-40-47-83-160 not have 2
/nfs3-p1/zsxm/dataset/2021-9-17-negative/yuyinglong-24-30-51-98 5 0.8
/nfs3-p1/zsxm/dataset/2021-9-17-negative/zhangmeiqin-19-22-39-86 5 0.8
/nfs3-p1/zsxm/dataset/2021-9-17-negative/zhouxiushan-14-18-34-84 5 0.8
/nfs3-p1/zsxm/dataset/2021-9-17-negative/zhumeizheng-34-42-69-167 2.500000 5.000000


In [8]:
for patient in not_2_list:
    print(patient)
print(len(not_2_list))

/nfs3-p1/zsxm/dataset/2021-9-17-negative/caiqishu-34-43-70-176
1


In [ ]:
root = '/nfs3-p1/zsxm/dataset/2021-10-19-imh'
wb = openpyxl.load_workbook(os.path.join(root, 'label.xlsx'))
sheet = wb['label']
for patient in sorted(os.listdir(root)):
    name = patient.split('-')[0]
    patient = os.path.join(root, patient)
    if os.path.isfile(patient):
        continue
    for row in sheet.iter_rows():
        if row[0].value == name:
            ct_label = row[3].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
            cta_label = row[4].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
            break
    print(name, ct_label, cta_label)
    ctl, ctal = ct_label.split('-'), cta_label.split('-')
    assert len(ctl) == len(ctal), name
    ct_start, ct_end, cta_start, cta_end = 10000, -1, 10000, -1
    ct_min_i, ct_max_i, cta_min_i, cta_max_i = -1, -1, -1, -1
    for i in range(len(ctl)):
        if int(ctl[i])-1 < ct_start:
            ct_start = int(ctl[i])-1
            ct_min_i = i
        if int(ctl[i])-1 > ct_end:
            ct_end = int(ctl[i])-1
            ct_max_i = i
        if int(ctal[i])-1 < cta_start:
            cta_start = int(ctal[i])-1
            cta_min_i = i
        if int(ctal[i])-1 > cta_end:
            cta_end = int(ctal[i])-1
            cta_max_i = i
        assert ct_min_i == cta_min_i, f'{len(ct)}  {len(cta)}'
        assert ct_max_i == cta_max_i, f'{len(ct)}  {len(cta)}'
    m, n = ct_end - ct_start, cta_end - cta_start
    assert m>0 and n>0
    print(m, n)

In [ ]:
def combine_image0(root):    
    wb = openpyxl.load_workbook(os.path.join(root, 'label.xlsx'))
    sheet = wb['label']
    size_not_match = set()
    for patient in sorted(os.listdir(root)):
        name = patient.split('-')[0]
        folder_name = patient
        patient = os.path.join(root, patient)
        if os.path.isfile(patient):
            continue
        ct, ct_thickness = load_scan(os.path.join(patient, '1'))
        cta, cta_thickness = load_scan(os.path.join(patient, '2'))
        for row in sheet.iter_rows():
            if row[0].value == name:
                ct_label = row[3].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
                cta_label = row[4].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
                break
        print(name, ct_label, cta_label)
        ctl, ctal = ct_label.split('-'), cta_label.split('-')
        assert len(ctl) == len(ctal), name
        ct_cta_table = [-len(cta)] * len(ct)
        if ct_thickness % cta_thickness == 0:
            times = int(ct_thickness // cta_thickness)
            ct_pivot, cta_pivot = int(ctl[0])-1, int(ctal[0])-1
            for i in range(len(ct)):
                ct_cta_table[i] = min(max(cta_pivot + (i - ct_pivot) * times, 0), len(cta)-1)
        else:
            ct_start, ct_end, cta_start, cta_end = int(ctl[0])-1, int(ctl[1])-1, int(ctal[0])-1, int(ctal[1])-1
            if ct_start > ct_end:
                ct_start, ct_end, cta_start, cta_end = ct_end, ct_start, cta_end, cta_start
            m, n = ct_end - ct_start, cta_end - cta_start
            ot, q = [], 0
            for p in range(m):
                min_dis = abs(q/n-p/m)
                while q < n:
                    q += 1
                    if abs(q/n-p/m) < min_dis:
                        min_dis = abs(q/n-p/m)
                    else:
                        q -= 1
                        ot.append(q)
                        break
            print(ot)
            assert len(ot) == m, name
            for i in range(len(ct)):
                ct_cta_table[i] = min(max(cta_start + (i-ct_start)//m*n + ot[(i-ct_start)%m], 0), len(cta)-1)
        #print(ct_cta_table)
        assert min(ct_cta_table) >=0 and max(ct_cta_table) < len(cta), f'{name}:{min(ct_cta_table)},{max(ct_cta_table)},{len(cta)}'
        image_path = os.path.join(patient, f'combine_images_{lower_b}_{upper_b}')
        if os.path.exists(image_path):
             shutil.rmtree(image_path)
        os.mkdir(image_path)
        for i in range(len(ct)):
            ct_img = ct[i].pixel_array.astype(np.int16)
            intercept = ct[i].RescaleIntercept
            slope = ct[i].RescaleSlope
            if slope != 1:
                ct_img = (slope * ct_img.astype(np.float64)).astype(np.int16)
            ct_img += np.int16(intercept)
            ct_img = np.clip(ct_img, lower_b, upper_b)
            ct_img = ((ct_img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)

            cta_img = cta[ct_cta_table[i]].pixel_array.astype(np.int16)
            intercept = cta[ct_cta_table[i]].RescaleIntercept
            slope = cta[ct_cta_table[i]].RescaleSlope
            if slope != 1:
                cta_img = (slope * cta_img.astype(np.float64)).astype(np.int16)
            cta_img += np.int16(intercept)
            cta_img = np.clip(cta_img, lower_b, upper_b)
            cta_img = ((cta_img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)

            if ct_img.shape[0] < cta_img.shape[0]:
                crop_start = int(round((cta_img.shape[0]-ct_img.shape[0])/2.))
                cta_img = cta_img[crop_start: crop_start+ct_img.shape[0]]
            if ct_img.shape[1] < cta_img.shape[1]:
                crop_start = int(round((cta_img.shape[1]-ct_img.shape[1])/2.))
                cta_img = cta_img[:, crop_start: crop_start+ct_img.shape[1]]
            if ct_img.shape[0] > cta_img.shape[0] or ct_img.shape[1] > cta_img.shape[1]:
                size_not_match.add(patient)
                break

            img = np.concatenate((ct_img, cta_img), axis=1)
            img = Image.fromarray(img)
            img.save(os.path.join(image_path, f'{folder_name}_{i:04d}_{ct_cta_table[i]:04d}.png'))
            
combine_image0('/nfs3-p1/zsxm/dataset/2021-10-19-imh')
combine_image0('/nfs3-p1/zsxm/dataset/2021-10-19-pau')

In [ ]:
def combine_image(root):    
    wb = openpyxl.load_workbook(os.path.join(root, 'label.xlsx'))
    sheet = wb['label']
    size_not_match = set()
    for patient in sorted(os.listdir(root)):
        name = patient.split('-')[0]
        folder_name = patient
        patient = os.path.join(root, patient)
        if os.path.isfile(patient):
            continue
        ct, ct_thickness = load_scan(os.path.join(patient, '1'))
        cta, cta_thickness = load_scan(os.path.join(patient, '2'))
        for row in sheet.iter_rows():
            if row[0].value == name:
                ct_label = row[3].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
                cta_label = row[4].value.replace('\n', ' ').replace(' ', '-').upper().replace('S', '').replace('J', '').replace('G', '')
                break
        print(name, ct_label, cta_label)
        ctl, ctal = ct_label.split('-'), cta_label.split('-')
        assert len(ctl) == len(ctal), name
        ct_cta_table = [-len(cta)] * len(ct)
        ct_start, ct_end, cta_start, cta_end = len(ct), -1, len(cta), -1
        ct_min_i, ct_max_i, cta_min_i, cta_max_i = -1, -1, -1, -1
        for i in range(len(ctl)):
            if int(ctl[i])-1 < ct_start:
                ct_start = int(ctl[i])-1
                ct_min_i = i
            if int(ctl[i])-1 > ct_end:
                ct_end = int(ctl[i])-1
                ct_max_i = i
            if int(ctal[i])-1 < cta_start:
                cta_start = int(ctal[i])-1
                cta_min_i = i
            if int(ctal[i])-1 > cta_end:
                cta_end = int(ctal[i])-1
                cta_max_i = i
            assert ct_min_i == cta_min_i and ct_max_i == cta_max_i
        m, n = ct_end - ct_start, cta_end - cta_start
        assert m > 0 and n > 0
        ot, q = [], 0
        for p in range(m):
            min_dis = abs(q/n-p/m)
            while q < n:
                q += 1
                if abs(q/n-p/m) < min_dis:
                    min_dis = abs(q/n-p/m)
                else:
                    q -= 1
                    ot.append(q)
                    break
        #print(ot)
        assert len(ot) == m, name
        for i in range(len(ct)):
            ct_cta_table[i] = min(max(cta_start + (i-ct_start)//m*n + ot[(i-ct_start)%m], 0), len(cta)-1)
        #print(ct_cta_table)
        assert min(ct_cta_table) >=0 and max(ct_cta_table) < len(cta), f'{name}:{min(ct_cta_table)},{max(ct_cta_table)},{len(cta)}'
        image_path = os.path.join(patient, f'combine_images_{lower_b}_{upper_b}')
        if os.path.exists(image_path):
             shutil.rmtree(image_path)
        os.mkdir(image_path)
        for i in range(len(ct)):
            ct_img = ct[i].pixel_array.astype(np.int16)
            intercept = ct[i].RescaleIntercept
            slope = ct[i].RescaleSlope
            if slope != 1:
                ct_img = (slope * ct_img.astype(np.float64)).astype(np.int16)
            ct_img += np.int16(intercept)
            ct_img = np.clip(ct_img, lower_b, upper_b)
            ct_img = ((ct_img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)

            cta_img = cta[ct_cta_table[i]].pixel_array.astype(np.int16)
            intercept = cta[ct_cta_table[i]].RescaleIntercept
            slope = cta[ct_cta_table[i]].RescaleSlope
            if slope != 1:
                cta_img = (slope * cta_img.astype(np.float64)).astype(np.int16)
            cta_img += np.int16(intercept)
            cta_img = np.clip(cta_img, lower_b, upper_b)
            cta_img = ((cta_img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)

            if ct_img.shape[0] < cta_img.shape[0]:
                crop_start = int(round((cta_img.shape[0]-ct_img.shape[0])/2.))
                cta_img = cta_img[crop_start: crop_start+ct_img.shape[0]]
            if ct_img.shape[1] < cta_img.shape[1]:
                crop_start = int(round((cta_img.shape[1]-ct_img.shape[1])/2.))
                cta_img = cta_img[:, crop_start: crop_start+ct_img.shape[1]]
            if ct_img.shape[0] > cta_img.shape[0] or ct_img.shape[1] > cta_img.shape[1]:
                size_not_match.add(patient)
                break

            img = np.concatenate((ct_img, cta_img), axis=1)
            img = Image.fromarray(img)
            img.save(os.path.join(image_path, f'{folder_name}_{i+1:04d}_{ct_cta_table[i]+1:04d}.png'))
            
combine_image('/nfs3-p1/zsxm/dataset/2021-10-19-imh')
combine_image('/nfs3-p1/zsxm/dataset/2021-10-19-pau')

In [ ]:
def move_together(root, dist):
    os.makedirs(dist, exist_ok=True)
    for patient in sorted(os.listdir(root)):
        if os.path.isfile(os.path.join(root, patient)):
            continue
        print(f'Processing {patient}')
        img_path = os.path.join(root, patient, f'combine_images_{lower_b}_{upper_b}')
        shutil.copytree(img_path, os.path.join(dist, patient))

move_together('/nfs3-p1/zsxm/dataset/2021-10-19-imh', '/nfs3-p1/zsxm/dataset/aorta_combine/imh')
move_together('/nfs3-p1/zsxm/dataset/2021-10-19-pau', '/nfs3-p1/zsxm/dataset/aorta_combine/pau')

In [ ]:
positive_folders = ['/nfs3-p2/zsxm/dataset/2021-9-8/','/nfs3-p2/zsxm/dataset/2021-9-13/','/nfs3-p2/zsxm/dataset/2021-9-19/','/nfs3-p2/zsxm/dataset/2021-9-28/']
patient_list = []
for folder in positive_folders:
    for patient in sorted(os.listdir(folder)):
        patient = os.path.join(folder, patient)
        if os.path.isdir(patient):
            patient_list.append(patient)
            
            
            
selected_list = ['baochunsheng-J-20-82', 'caiweiguang-J-Im35-152', 'chenbo-S-Im24-45-J-Im24-97', 'chenfujun-J-Im43-100', 'chengshizheng-J-Im30-145', 'chengyonghui-J-17-81', 'chenjiandong-S-19-28-J-19-91', 'chenping-J-Im23-88', 'chensiqi-S-17-28-J-17-69', 'chugentang-J-Im19-80', 'fangchunfeng-J-16-48', 'fanghongwei-J-22-71', 'guanxujun-S-Im25-35-J-Im25-87', 'guoheying-J-Im32-127', 'guquankang-J-Im18-69', 'huangdebing-S-Im40-48-J-Im33-138', 'huanghai-S-Im19-29-J-Im19-80', 'huangsuyue-J-Im21-41', 'huangxihong-J-Im27-136', 'huangyong-J-Im18-127', 'huangyongliang-J-Im16-81', 'jiadeen-J-Im34-148', 'jihongke-S-Im20-24-J-Im20-86', 'laiguizhen-J-Im38-121', 'laixuexiang-J-Im33-148', 'lanshaomei-S-Im20-31-J-Im20-79', 'leixiaoying-S-Im18-26-J-18-71', 'lihuanhuan-S-Im20-28-J-Im20-59', 'liubihai-S-23-26-J-23-93', 'liyehua-S-Im30-47-J-Im30-131', 'louyidong-S-Im38-58-J-Im38-133', 'luojun-J-Im22-81', 'maliwei-S-Im30-48-J-Im30-101', 'pengzhengjiang-J-Im38-160', 'qumin-S-Im22-35-J-Im22-28', 'ruweiping-S-Im52-63-J-Im52-94', 'shaoyefeng-S-Im34-43-J-Im34-48', 'shenglina-J-Im21-65', 'shenqi-J-Im24-72', 'tangabiao-S-Im41-49-J-Im41-138', 'wangjiangwei-J-Im37-143', 'wangxiaofu-S-19-29-J-19-65', 'wangyonghui-S-Im18-27-J-Im18-84', 'wangyuefeng-S-Im34-38-J-Im34-144', 'wangzhengjin-J-16-82', 'wenyongguo-J-Im23-121', 'wuyueming-J-Im36-138', 'xiaminsong-S-Im40-51-J-Im40-139', 'xiashubiao-J-24-88', 'xuhangying-J-25-63', 'xuyaofeng-J-Im24-119', 'xuzhaofang-S-21-26-J-21-86', 'yangmin-J-Im37-150', 'yangxuehua-J-Im23-79', 'yangzhanxiang-J-Im31-145', 'yepeng-J-Im26-125', 'yexianchang-S-17-30-J-17-74', 'yexiyou-J-Im35-139', 'yujiada-J-Im21-48', 'zhangguangming-J-21-138', 'zhanghaitao-S-Im25-49-J-Im25-145', 'zhangwei-S-20-26-J-20-61', 'zhangyunqin-S-Im14-25-J-Im14-64', 'zhaowenxian-J-Im43-111', 'zhengguozheng-J-18-81', 'zhengjiyou-J-Im28-136', 'zhoufeng-S-Im35-40-J-Im35-147', 'zhouliang-J-Im47-123', 'zhuseng-S-Im24-31-J-Im24-105', 'zhuxutao-J-Im23-78', 'zhuyongfu-J-Im22-80', 'zhuyuejin-J-Im21-72', 'zongminghui-S-Im18-25-J-Im18-81']+['chenggang-J-Im18-81', 'chenjufa-J-16-66', 'chenjuli-J-Im18-49', 'chenjun-J-Im33-137', 'chenlili-S-Im18-24-J-Im18-74', 'chiyanfei-J-Im20-83', 'daizuokou-J-Im19-74', 'ganxiaobin-J-23-51', 'guojianfu-J-Im32-100', 'hongzhimin-S-Im30-39-J-30-144', 'huanglijun-J-24-98', 'huleijun-J-Im25-141', 'jilixian-J-Im34-108', 'jinchongfei-J-22-92', 'lijianming-J-Im32-124', 'lingenqiang-S-Im30-39', 'linjiaxiang-J-21-72', 'liuyunfei-J-Im40-95', 'loulinhua-S-Im27-30', 'lufeng-J-Im28-103', 'luzhiping-S-Im25-35-J-Im25-54', 'panzhangsong-J-Im26-91', 'qianfuying-J-Im20-87', 'qiaowei-S-Im18-30-J-Im18-82', 'shanghongjun-J-22-84', 'shaoxiulan-J-Im21-25', 'shengwenping-S-Im37-46-J-Im37-116', 'shenliqiang-J-Im29-137', 'shenxuefu-S-Im33-40-J-Im33-101', 'sishouzhong-J-Im25-137', 'wangqing-J-35-102', 'wutonggen-S-Im22-29-J-Im22-82', 'wuwanglong-J-Im31-125', 'wuxiangyang-S-Im18-25', 'xiafangzhou-S-18-25-J-18-80', 'xuhong-S-Im23-38-J-Im23-40', 'xushichao-J-Im35-145', 'yangbingshui-S-Im25-33-J-Im25-56', 'yanjuanfeng-J-Im30-32', 'yaocaiming-J-Im26-88', 'yaojianmin-S-Im20-29-J-Im20-86', 'yesanghua-J-Im41-138', 'yewenyi-J-Im36-130', 'yingguoliang-J-Im55-130', 'yingjianquan-S-Im12-14-J-Im12-36', 'yingmeiqi-S-Im22-25-J-Im16-36', 'yintianxing-J-Im24-38', 'yuanlinyue-J-Im27-120', 'yuguiying-S-16-22-J-16-77', 'yuhongliang-S-Im26-37-J-Im26-105', 'yujiazhen-S-Im19-27-J-Im19-77', 'zhangboqian-S-Im22-32-J-Im22-88', 'zhangfuyang-S-Im15-20-J-Im15-87', 'zhangjian-J-26-89', 'zhanglimin-J-Im40-149', 'zhongxuefang-S-Im19-34-J-Im19-72', 'zhoubozhong-J-Im29-131', 'zhoumingfang-J-Im36-41', 'zhousuhua-J-Im25-96', 'zhuhancheng-S-Im27-36-J-Im27-86']
processed_list = []
for patient in patient_list:
    if patient.split('/')[-1] in selected_list:
        print(patient)
        processed_list.append(patient)
        

In [ ]:
ksize = 1
print(isinstance(ksize, int))

In [ ]:
a = [1]*10
print(a)

In [ ]:
a[1] = 7
print(a)